In [20]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind
import dc_stat_think as dcst

#Import data
df = pd.read_csv('marketing_new.csv')

# Setting up the data -- slice the channel of interest
email = df[df['marketing_channel'] == 'Email']

# Group by user_id and variant (control and personalised)
subscribers = email.groupby(['user_id', 'variant'])['converted'].max()

# Convert array to dataframe
subscribers = pd.DataFrame(subscribers.unstack(level=1))

# Drop missing values
control = subscribers['control'].dropna()
personalised = subscribers['personalization'].dropna()

# Convert control and personalised arrays to numeric data type for the ttest_ind() function
control = control.astype(float)
personalised = personalised.astype(float)

# Calculate the conversion rate
control_mean = np.mean(control)
personalised_mean = np.mean(personalised)

# Compute the difference of means
difference = personalised_mean - control_mean

#print results
print('Conversion rate control group: ', control_mean, '\nConversion rate personalised group: ', personalised_mean)
print('Differnce in conversion rate: ', difference )

# Is this difference statistically significant? Calculate Lift to further investigate
# METHOD 1 
lift = difference/control_mean
print('lift:', str(round(lift*100, 2)) + '%')

# Conduct a two-sample t-test
t = ttest_ind(personalised, control)
print(t)
# Result: reject null hypothesis

Conversion rate control group:  0.2814814814814815 
Conversion rate personalised group:  0.3908450704225352
Differnce in conversion rate:  0.10936358894105369
lift: 38.85%
Ttest_indResult(statistic=2.734329944750507, pvalue=0.006451487844694182)


In [19]:
# METHOD 2
# Use permutation Test Method to conduct A/B Test on the same data
def diff_frac(data_A, data_B):
    frac_A = np.sum(data_A)/len(data_A)
    frac_B = np.sum(data_B)/len(data_B)
    return frac_A - frac_B

# Compute observed test statistic
diff_frac_obs = diff_frac(personalised, control)

# Functions
# Inner function
def permutation_sample(data_1, data_2):
    data = np.concatenate((data_1, data_2))
    permutted_data = np.random.permutation(data)
    perm_sample_1 = permutted_data[:len(data_1)]
    perm_sample_2 = permutted_data[len(data_1):]
    return perm_sample_1, perm_sample_2

# Outer function
def draw_perm_reps(data_1, data_2, func, size=1):
    perm_replicates = np.empty(size)
    for i in range(size):
        perm_sample_1, perm_sample_2 = permutation_sample(data_1, data_2)
        perm_replicates[i] = func(perm_sample_1, perm_sample_2)
    return perm_replicates

# Acquire 10000 replicates
perm_replicates = np.empty(10000)
for i in range(10000):
    perm_replicates[i] = draw_perm_reps(personalised, control, diff_frac)

p_value = np.sum(perm_replicates >= diff_frac_obs)/10000

# Display output
print('Differnce in conversion rate: ', diff_frac_obs, '\nP_value', p_value, '\nResult: Reject the null hypothesis')

# Result: reject null hypothesis

# Despite the slight difference in p-values, both tests yield statistically significant results considering the threshold of 0.05. 
# This indicates strong evidence against the null hypothesis and supports the conclusion of a significant difference between the groups.

Differnce in conversion rate:  0.10936358894105369 
P_value 0.0036 
Result: Reject the null hypothesis


In [18]:
# METHOD 3 -- using the formula for calculating the t-statistic, 
# and looking up the p-value in the t-Distribution Critical Values Table
s1 = np.std(control)
s2 = np.std(personalised)

n1 = len(control)
n2 = len(personalised)

test_stat = difference/ np.sqrt((s1**2/n1) + (s2**2/n2))

degrees_of_freedom = (n1 - 1) + (n2 - 1)
print('Test Statistic is the same value as was returned by the ttest_ind() function in Method 1: ', test_stat,
      '\nThe p-value falls between 0.005 and 0.0025, so we do reject the null hypothesis as was the case in Methods 1 and 2.')

# p-value according to the table falls between .005 and .0025

Test Statistic is the same value as was returned by the ttest_in() function in Method 1:  2.744918360261471 
The p-value falls between 0.005 and 0.0025, so we do reject the null hypothesis as was the case in Methods 1 and 2.
